<a href="https://colab.research.google.com/github/firststef/APLN-WSD/blob/master/wsd_ourData2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy sklearn rowordnet
!python -m spacy download ro_core_news_lg
!pip install bs4
!pip install sparknlp pyspark

In [ ]:
from bs4 import BeautifulSoup
import spacy

nlp = spacy.load("ro_core_news_lg")


def parse_xml(path):
  with open(path, encoding='utf8') as fp:
    file_p = fp.read()
    soup = BeautifulSoup(file_p, 'html.parser')
  data = soup.find_all('instance')
  parsed_data = {}
  for sense in data:
    new_data = []
    contexts = sense.find_all('context')
    senseId = sense.attrs["id"][10:]
    if len(senseId)<=1:
      continue
    word = ''
    for context in contexts:
      try:
        word = nlp(context.head.get_text())[0].lemma_
        sentence = context.decode_contents().strip('\n')

        #print(word)
        a, *b = sentence.split('<head>')
        b, *c = ''.join(b).split('</head>')
        sentence = a + b + ''.join(c)
        sentence= ' '.join(sentence.split())
        new_data.append((sentence, senseId))
        
      except Exception as e:
        print(e)
        continue
    if word =='':
      continue
    if word in parsed_data:
      parsed_data[word].extend(new_data)
    else:
      parsed_data[word] = new_data
     
  return parsed_data



prep_data = parse_xml("/content/drive/MyDrive/Colab Notebooks/wsd/toate.xml")
print(len(prep_data.keys()))
print(prep_data["lac"][0])



In [ ]:
import random
def split_data(data):
  train_data = {}
  test_data = {}
  for word, vector in data.items():
    senses = {}
    for tup in vector:
      context, sense = tup
      if sense not in senses:
        senses[sense] = []
      senses[sense].append(tup)

    train_data[word] = []
    test_data[word] = []

    # print(senses)

    for sense in senses:
      vec = senses[sense]
      l = len(vec)
      marg = 2*l//3
      if l == 1:
        train_data[word].extend(vec[0])
        test_data[word].extend(vec[0])
      else:
        random.shuffle(vec)
        ex_train = vec[:marg]
        ex_test = vec[marg:]
        train_data[word].extend(ex_train)
        test_data[word].extend(ex_test)
  return train_data, test_data

train_data, test_data = split_data(prep_data)
print(train_data["lac"])
print(test_data["lac"])

In [ ]:
import nltk
import spacy
from sklearn.feature_extraction import DictVectorizer
import rowordnet as rwn
from nltk.classify import MaxentClassifier

vec = DictVectorizer()
wn = rwn.RoWordNet()
nlp = spacy.load("ro_core_news_lg")


def extract_hypernymes(word):
    hyper = []
    synset_ids = wn.synsets(literal=word.lemma_, pos=rwn.synset.Synset.Pos.NOUN)
    if len(synset_ids) > 0:
        c = wn.synset_to_hypernym_root(synset_ids[0])
        for c1 in c:
            if len(wn(c1).literals):
                hyper.append(wn(c1).literals[0])
    hyper_dict = {}
    for i in range(len(hyper)):
      hyper_dict["hyper"+str(i)] = hyper[i]
    #print(hyper_dict)
    return hyper_dict


def extract_feat_for_word(word, rel):
    hyper = {}
    if word.pos_ == "NOUN":
        hyper = extract_hypernymes(word)
    d = {
        ("text"+rel): word.text,
        "pos"+rel: word.pos_,
        "morpho"+rel: word.morph,
        "lemma"+rel: word.lemma_,
        "dep"+rel: word.dep_,
         "ner"+rel: word.ent_type_,
        #"hyper"+rel: hyper,
        #"position_rel": rel
    }
    d.update(hyper)
    return d

def create_features_vector(word, context):
    text = nlp(context)
    vect = {}
    for w in text:
      if w.lemma_ == word:
            if w.head != w:
              
              #print(w.head)
              vect.update(extract_feat_for_word(w.head, "head"))
              for ind, siblings in enumerate(w.head.children):
                vect.update(extract_feat_for_word(siblings, "0_"+str(ind) if w != siblings else "main"))
            else:
              vect.update(extract_feat_for_word(w, "main"))
            for chil in w.children:
              vect.update(extract_feat_for_word(chil, "children"))
        
    return vect

def prepare_data(data):
  new_data_per_words = {}

  for word, instances in data.items():
    sense = word
    #print(sense)
    new_data = []
    #print(instances)
    
    for i in instances:
      #print(i)
      #print(i[0])
      new_data.append((create_features_vector(sense,i[0]), i[1]))
    new_data_per_words[word] = new_data


  return new_data_per_words

a = prepare_data(train_data)

#test_data, test_labels = prepare_test_data(test_se3)
b = prepare_data(test_data)
print(a["lac"])
print(b["lac"])
print(a.keys())

In [ ]:


algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifiers = {}
for w in a.keys():
  print(w)
  classifier = nltk.MaxentClassifier.train(a[w], algorithm, max_iter=20)
  #classifier.show_most_informative_features(40, show='all')
  #print(classifier.weights())
  #print(feat)
  classifiers[w] = classifier

all_accs = []
for w in b.keys():
  output_labels = classifiers[w].classify_many([x[0] for x in b[w]])
  acc = nltk.classify.accuracy( classifiers[w], b[w])
  all_accs.append(acc)

print("Precizie", avg(all_accs))

In [ ]:

import rowordnet as rwn
wn = rwn.RoWordNet()
a = wn('ENG30-00982602-a').literals
print(a)

In [ ]:
import spacy
nlp = spacy.load("ro_core_news_lg")
title = nlp('mării')
print(title[0].lemma_)
